In [93]:
import csv
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats

import bokeh.io
import bokeh.plotting
import bokeh.palettes
import bokeh.models

bokeh.io.output_notebook()

Loading BokehJS ...

#### Want to calibrate and convert the PCA fluorescence into a concentration

In [249]:
def general_hill(x, Ka, n, A, B, C):
    """
    Use a generalized hill function for the calibration curve.
    """
    
    y = B + A / (C + (Ka / x) ** n)
    
    return y

def inverse_general_hill(y, Ka, n, A, B, C):
    """
    Inverse function for deriving concentrations from fluorescence values.
    """
    
    x = Ka / ((A / (y - B) - C) ** (1/n))
    
    return x

def get_calib_data(fluor_df):
    """
    Function to extract calibration data from the general dataframe(s).
    """
    
    calib_df = fluor_df.loc[fluor_df['Strain'] == 'calibration']
    calib_df['Condition Conc. (µM)'] = calib_df['Condition Conc. (µM)'].astype(float)
    
    return calib_df

def plot_calib_point(calib_df, title=None):
    """
    Plotter for the calibration data.
    """
    
    
    fig = bokeh.plotting.figure(height=400, width=600, title=title)
    
    c = fig.circle(calib_df['Condition Conc. (µM)'], calib_df['PCAred fluorescence (AU)'], size=5, alpha=0.05, legend='Calibration measurements')
    
    fig.legend.location = 'bottom_right'
    
    return fig
    
def fit_hill(calib_df):
    """
    Function to fit the generalized Hill function to the calibration data.
    """
    
    xdata = calib_df['Condition Conc. (µM)'].values
    ydata = calib_df['PCAred fluorescence (AU)'].values
    
    popt, pcov = scipy.optimize.curve_fit(general_hill, xdata, ydata, p0 = [150, 2, 9000, 1000, 1])
    
    plot = plot_calib_point(calib_df, title='Fit of calibration model')
    
    x = np.linspace(0, 800, 100)
    fit = general_hill(x, *popt)
    
    plot.line(x, fit, color='black')
    
    bokeh.io.show(plot)
    
    return popt, pcov

def convert_fluor_to_conc(fluor_exp_df, popt):
    """
    Function to convert fluorescence measurements to concentrations.
    """
    
    fluor_exp_df['measured PCAred (µM)'] = [inverse_general_hill(f, *popt) for f in fluor_exp_df['PCAred fluorescence (AU)']]
    
    return fluor_exp_df

def fitting_pipeline(df):
    """
    Function to bring above utilities together.
    """
    
    calib_df = get_calib_data(df)
    exp_df = df.loc[df['Strain'] != 'calibration']
    
    popt, pcov = fit_hill(calib_df)
    
    exp_df = convert_fluor_to_conc(exp_df, popt)
    
    return exp_df

def linear_approximation(x_array, y_array, cutoff_time):
    
    indices = x_array < cutoff_time
    
    x = x_array[indices]
    y = y_array[indices]
    
    slope, intercept, rvalue, pvalue, stderr = scipy.stats.linregress(x, y)
    
    return slope, intercept, rvalue

def linear_range_evaluator(single_strain_single_condition_df, y_var, title=None):
    
    x_array = single_strain_single_condition_df['Time [hr]'].values
    y_array = single_strain_single_condition_df[y_var].values
    
    cutoff_times = np.linspace(0, 24, 48)
    
    r_squareds = []
    slopes = []
    ints = []
    
    for ct in cutoff_times:
        
        try:
            s, i, r = linear_approximation(x_array, y_array, ct)

            r_squareds.append(r**2)
            slopes.append(s)
            ints.append(i)
            
        except:
            r_squareds.append(None)
            slopes.append(None)
            ints.append(None)
            
    
    source = bokeh.models.ColumnDataSource(data=dict(cutoff_times=cutoff_times,
                                                     r_squareds=r_squareds,
                                                     slope_int=[f'slope: {s}\nintercept:{i}' for s, i in zip(slopes, ints)])
                                          )
        
    fig = bokeh.plotting.figure(width=200,
                                height=200,
                                x_axis_label='Cutoff time (hours)',
                                y_axis_label='R-squared',
                                title=title)
    
    c = fig.circle(x='cutoff_times', y='r_squareds', source=source)
    
#     labels = bokeh.models.LabelSet(x='cutoff_times', y='r_squared', text='slope_int',
#                                    x_offset=5, y_offset=5, source=source)
    
#     fig.add_layout(labels)
    
    fig.y_range = bokeh.models.Range1d(0, 1.05)
    fig.output_backend = 'svg'
    
    return fig

def linear_assessment_plotter(df, y_var='measured PCAred (µM)'):
    """
    Function to plot R**2 against cutoff times to determine a linear range to estimate
    initial rates.
    """
    
    grouped = df.groupby(['Strain', 'Condition'])
    
    plots = []
    
    for g in grouped:
        strain = g[0][0]
        condition = g[0][1]
        mini_df = g[1]
        
        mini_df = mini_df.dropna()
        
        title = f'{strain} with {condition} linearity'
        
#         print(strain, condition)
        p = linear_range_evaluator(mini_df, y_var, title)
        
        plots.append(p)
        
    return plots

def plotter(df, 
            plot_grouping, 
            color_grouping, 
            y='measured PCAred (µM)', 
            y_axis_label='Reduced PCA (µM)', 
            x_axis_label='Time (hrs)'):
    
    """
    Function to plot the data for paper figures.
    
    plot_grouping and color_grouping are either "Condition" or "Strain"
    """
    
    plot_grouped = df.groupby(plot_grouping)
    
    plots = []
    
    for grp in plot_grouped:
        
        title = f"{plot_grouping}: {grp[0]}"
        
        fig = bokeh.plotting.figure(
                width=600,
                height=300,
                title=title,
                y_axis_label=y_axis_label,
                x_axis_label=x_axis_label
            )
        
        mini_df = grp[1]
        
        color_grouped = mini_df.groupby(color_grouping)
        
        palette = list(bokeh.palettes.Colorblind6)
        palette[0] = 'grey'
        
        legend_items = []
        
        for i, g in enumerate(color_grouped):
            
            label = g[0]
            
            mdf = g[1]
            
            try:
                wells = mdf['Well'].unique()
            
            except:
                wells = []
            
            if len(wells) > 1: # Need to account for replicates
                
                measurement_arrays = []
                cs = []
                for w in wells:
                    time = mdf.loc[mdf['Well'] == w]['Time [hr]'].values
                    measurement = mdf.loc[mdf['Well'] == w][y].values
                    
                    measurement_arrays.append(measurement)
                    
                    c = fig.circle(time, measurement, color = palette[i], alpha=0.2, size=2)
                    cs.append(c)
                    
                mean = sum(measurement_arrays) / len(measurement_arrays)
                
                l = fig.line(time, mean, color = palette[i], alpha=1, line_width=3)
                
                legend_items.append((label, [l, *cs]))
#                 print(legend_items)
                
            else:
                time = mdf['Time [hr]'].values
                measurement = mdf[y].values
                
                c = fig.circle(time, measurement, color = palette[i], alpha=0.7)
                
                legend_items.append((label, [c]))
                
            if plot_grouping == 'Strain' and y == 'measured PCAred (µM)':
                
                if label != 'PCA, NO2':
                    lin_reg_df = mdf.loc[mdf['Time [hr]'] <= 5]
                else:
                    lin_reg_df = mdf.loc[(mdf['Time [hr]'] > 1.5) & (mdf['Time [hr]'] <= 5)]
                    
                lin_reg_df = lin_reg_df.dropna()
                
                if len(lin_reg_df) == 0:
                    
                    print(f"""
                    {grp[0]}
                    ------
                    {label}: no detectable redox
                    """)
                else:    
                    slope, inter, r, p, e = scipy.stats.linregress(lin_reg_df['Time [hr]'].values,
                                                           lin_reg_df[y].values)

    #                 print(lin_reg_df['Time [hr]'].values, lin_reg_df[y].values)

                    lin_x = np.linspace(0,24,5)
                    lin_y = slope * lin_x + inter


                    reg_l = fig.line(lin_x, lin_y, color = palette[i], line_dash='dashed', line_width=2, alpha=1)
                    print(f"""
                    {grp[0]}
                    ------
                    {label}: init redox rate {slope:.2f} µM PCA per hour with stderr: {e:.2f}
                    """)
        
        if y == 'measured PCAred (µM)':
            fig.y_range = bokeh.models.Range1d(-5, 205)
            
        elif y == 'OD600' or y == 'Mean OD600':
            fig.y_range = bokeh.models.Range1d(0, 0.3)
        
        legend = bokeh.models.Legend(items=legend_items)
        legend.click_policy = "hide"
        
        fig.add_layout(legend, 'right')
        
        fig.legend.label_text_font_style = "italic"
        fig.legend.label_text_font_size = '12pt'
        fig.title.text_font_size = "14pt"
        
        fig.yaxis.axis_label_text_font_size = '12pt'
        fig.xaxis.axis_label_text_font_size = '12pt'
        
        fig.yaxis.major_label_text_font_size = '10pt'
        fig.xaxis.major_label_text_font_size = '10pt'
        
        fig.output_backend = 'svg'
        
        plots.append(fig)
        
        for p in plots[1:]:
            p.x_range = plots[0].x_range
            p.y_range = plots[0].y_range
            
    return plots
    
    

## Notebook to generate figures for _C. portucalensis_ MBL Observation paper for mBio

In [3]:
reduction_df = pd.read_csv('./data/reduction_assay_dataframe.csv')
oxidation_df = pd.read_csv('./data/oxidation_assay_dataframe.csv')

In [4]:
oxidation_df = fitting_pipeline(oxidation_df)
oxidation_df

/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in power
  


/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Time [hr],Well,PCAred fluorescence (AU),Strain,Medium,Condition,Condition Conc. (µM),OD600,measured PCAred (µM)
3468,0.061,B1,5796,C. portucalensis MBL,basal medium,PCA,200,0.096,168.579375
3469,0.144,B1,5815,C. portucalensis MBL,basal medium,PCA,200,0.098,169.728620
3470,0.228,B1,5835,C. portucalensis MBL,basal medium,PCA,200,0.098,170.947811
3471,0.311,B1,5802,C. portucalensis MBL,basal medium,PCA,200,0.100,168.941355
3472,0.394,B1,5806,C. portucalensis MBL,basal medium,PCA,200,0.101,169.183156
...,...,...,...,...,...,...,...,...,...
24271,23.728,G12,5989,Abiotic,basal medium,"PCA, Fum","200, 10000",0.076,180.675627
24272,23.811,G12,5975,Abiotic,basal medium,"PCA, Fum","200, 10000",0.076,179.765455
24273,23.894,G12,5986,Abiotic,basal medium,"PCA, Fum","200, 10000",0.076,180.480140
24274,23.978,G12,5977,Abiotic,basal medium,"PCA, Fum","200, 10000",0.076,179.895152


In [5]:
reduction_df = fitting_pipeline(reduction_df)
reduction_df

/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in power
  


/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/Users/lmt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Time [hr],Well,PCAred fluorescence (AU),Strain,Medium,Condition,Condition Conc. (µM),OD600,measured PCAred (µM)
3468,0.061,B1,753,C. portucalensis MBL,basal medium,PCA,200,0.102,NaN
3469,0.144,B1,741,C. portucalensis MBL,basal medium,PCA,200,0.104,NaN
3470,0.228,B1,742,C. portucalensis MBL,basal medium,PCA,200,0.106,NaN
3471,0.311,B1,760,C. portucalensis MBL,basal medium,PCA,200,0.106,NaN
3472,0.394,B1,773,C. portucalensis MBL,basal medium,PCA,200,0.107,NaN
...,...,...,...,...,...,...,...,...,...
24271,23.728,G12,681,Abiotic,basal medium,"PCA, Fum","200, 10000",0.075,NaN
24272,23.811,G12,688,Abiotic,basal medium,"PCA, Fum","200, 10000",0.075,NaN
24273,23.894,G12,689,Abiotic,basal medium,"PCA, Fum","200, 10000",0.076,NaN
24274,23.978,G12,693,Abiotic,basal medium,"PCA, Fum","200, 10000",0.076,NaN


In [250]:
oxidation_plots_by_condition = plotter(oxidation_df, 'Condition', 'Strain')

In [251]:
oxidation_plots_by_strain = plotter(oxidation_df, 'Strain', 'Condition')


                    Abiotic
                    ------
                    PCA: init redox rate 1.39 µM PCA per hour with stderr: 0.17
                    

                    Abiotic
                    ------
                    PCA, Fum: init redox rate 1.43 µM PCA per hour with stderr: 0.33
                    

                    Abiotic
                    ------
                    PCA, NO2: init redox rate -1.45 µM PCA per hour with stderr: 0.16
                    

                    Abiotic
                    ------
                    PCA, NO3: init redox rate 1.27 µM PCA per hour with stderr: 0.23
                    

                    C. portucalensis MBL
                    ------
                    PCA: init redox rate 1.43 µM PCA per hour with stderr: 0.08
                    

                    C. portucalensis MBL
                    ------
                    PCA, Fum: init redox rate -6.51 µM PCA per hour with stderr: 0.26
                    

         

In [252]:
reduction_plots_by_condition = plotter(reduction_df, 'Condition', 'Strain')

In [253]:
reduction_plots_by_strain = plotter(reduction_df, 'Strain', 'Condition')


                    Abiotic
                    ------
                    PCA: no detectable redox
                    

                    Abiotic
                    ------
                    PCA, Fum: no detectable redox
                    

                    Abiotic
                    ------
                    PCA, NO2: no detectable redox
                    

                    Abiotic
                    ------
                    PCA, NO3: no detectable redox
                    

                    C. portucalensis MBL
                    ------
                    PCA: init redox rate 5.42 µM PCA per hour with stderr: 0.04
                    

                    C. portucalensis MBL
                    ------
                    PCA, Fum: init redox rate 3.04 µM PCA per hour with stderr: 0.09
                    

                    C. portucalensis MBL
                    ------
                    PCA, NO2: init redox rate 4.35 µM PCA per hour with stderr: 0.1

In [254]:
bokeh.io.show(oxidation_plots_by_condition[2])

In [255]:
bokeh.io.show(oxidation_plots_by_strain[1])

In [256]:
bokeh.io.show(reduction_plots_by_condition[0])

In [257]:
bokeh.io.show(reduction_plots_by_strain[1])

In [156]:
bokeh.io.export_svgs(oxidation_plots_by_condition, filename='./plots/oxidation/oxidation_plots_by_condition.svg')
bokeh.io.export_svgs(oxidation_plots_by_strain, filename='./plots/oxidation/oxidation_plots_by_strain.svg')

bokeh.io.export_svgs(reduction_plots_by_condition, filename='./plots/reduction/reduction_plots_by_condition.svg')
bokeh.io.export_svgs(reduction_plots_by_strain, filename='./plots/reduction/reduction_plots_by_strain.svg')

['./plots/reduction/reduction_plots_by_strain.svg',
 './plots/reduction/reduction_plots_by_strain_1.svg',
 './plots/reduction/reduction_plots_by_strain_1_2.svg',
 './plots/reduction/reduction_plots_by_strain_1_2_3.svg',
 './plots/reduction/reduction_plots_by_strain_1_2_3_4.svg',
 './plots/reduction/reduction_plots_by_strain_1_2_3_4_5.svg']

In [159]:
ox_linearity_plots = linear_assessment_plotter(oxidation_df)

In [160]:
bokeh.io.show(bokeh.layouts.gridplot(ox_linearity_plots, ncols=4))

In [161]:
bokeh.io.export_svgs(ox_linearity_plots, filename='./plots/linearity/oxidation_Rsquared_vs_cutoff.svg')

['./plots/linearity/oxidation_Rsquared_vs_cutoff.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6_7.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10_11.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10_11_12.svg',
 './plots/linearity/oxidation_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10_11_12_13.svg',
 './plots/linearity/oxidation_Rsquared

In [162]:
red_linearity_plots = linear_assessment_plotter(reduction_df)
bokeh.io.show(bokeh.layouts.gridplot(red_linearity_plots, ncols=4))

/Users/lmt/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/lmt/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/lmt/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Users/lmt/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/Users/lmt/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/lmt/miniconda3/lib/python3.7/site-packag

In [163]:
bokeh.io.export_svgs(red_linearity_plots, filename='./plots/linearity/reduction_Rsquared_vs_cutoff.svg')

['./plots/linearity/reduction_Rsquared_vs_cutoff.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6_7.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10_11.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10_11_12.svg',
 './plots/linearity/reduction_Rsquared_vs_cutoff_1_2_3_4_5_6_7_8_9_10_11_12_13.svg',
 './plots/linearity/reduction_Rsquared

In [11]:
oxidation_OD_by_condition = plotter(oxidation_df, 'Condition', 'Strain', y='OD600', y_axis_label='OD600')

In [12]:
bokeh.io.show(oxidation_OD_by_condition[0])

In [13]:
oxidation_OD_by_strain = plotter(oxidation_df, 'Strain', 'Condition', y='OD600', y_axis_label='OD600')

In [20]:
bokeh.io.show(oxidation_OD_by_strain[5])

#### Load data for ion chromatography

In [12]:
ic_df = pd.read_csv('./data/ion_chromatography_tidy.csv')
ic_df.head()

,Sample ID,timepoint,Nitrite (mM),Nitrate (mM),Acetate (mM),row,column,ace,pca,strain,Time (hours)
0,B1,T0,0.0777,10.3053,0.0552,B,1,0,0 µM PCA,C. portucalensis MBL,0.0
1,C1,T0,0.0455,10.5099,0.0328,C,1,0,0 µM PCA,C. portucalensis MBL,0.0
2,D1,T0,0.0695,10.8882,0.0187,D,1,0,0 µM PCA,C. portucalensis MBL,0.0
3,B4,T0,0.0775,10.3049,0.0543,B,4,0,200 µM PCAox,C. portucalensis MBL,0.0
4,C4,T0,0.0844,10.6758,0.0196,C,4,0,200 µM PCAox,C. portucalensis MBL,0.0


In [13]:
abio_ic_df = ic_df.loc[ic_df['row'] == 'E']
bio_ic_df = ic_df.loc[ic_df['row'] != 'E']

In [14]:
bio_ic_df

,Sample ID,timepoint,Nitrite (mM),Nitrate (mM),Acetate (mM),row,column,ace,pca,strain,Time (hours)
0,B1,T0,0.0777,10.3053,0.0552,B,1,0,0 µM PCA,C. portucalensis MBL,0.0
1,C1,T0,0.0455,10.5099,0.0328,C,1,0,0 µM PCA,C. portucalensis MBL,0.0
2,D1,T0,0.0695,10.8882,0.0187,D,1,0,0 µM PCA,C. portucalensis MBL,0.0
3,B4,T0,0.0775,10.3049,0.0543,B,4,0,200 µM PCAox,C. portucalensis MBL,0.0
4,C4,T0,0.0844,10.6758,0.0196,C,4,0,200 µM PCAox,C. portucalensis MBL,0.0
...,...,...,...,...,...,...,...,...,...,...,...
103,C6,T3,11.4206,0.0078,46.9181,C,6,50,200 µM PCAox,C. portucalensis MBL,53.0
104,D6,T3,11.2461,0.0145,47.5036,D,6,50,200 µM PCAox,C. portucalensis MBL,53.0
105,B9,T3,11.2677,0.0154,47.0770,B,9,50,200 µM PCAred,C. portucalensis MBL,53.0
106,C9,T3,11.4955,0.0086,48.1258,C,9,50,200 µM PCAred,C. portucalensis MBL,53.0


Get 95% confidence intervals

In [15]:
grouped = bio_ic_df.groupby(['Time (hours)', 'strain', 'pca', 'ace'])

times = []
strains = []
pcas = []
aces = []

no3_means = []
no2_means = []
ace_means = []

no3_standard_errors = []
no2_standard_errors = []
ace_standard_errors = []

for g in grouped:
    
    times.append(g[0][0])
    strains.append(g[0][1])
    pcas.append(g[0][2])
    aces.append(g[0][3])
    
    no3_means.append(np.mean(g[1]['Nitrate (mM)']))
    no3_standard_errors.append(np.std(g[1]['Nitrate (mM)'])/np.sqrt(3))
    
    no2_means.append(np.mean(g[1]['Nitrite (mM)']))
    no2_standard_errors.append(np.std(g[1]['Nitrite (mM)'])/np.sqrt(3))
    
    ace_means.append(np.mean(g[1]['Acetate (mM)']))
    ace_standard_errors.append(np.std(g[1]['Acetate (mM)'])/np.sqrt(3))

no3_stat_df = pd.DataFrame.from_dict({'strain': strains,
                                      'Time (hours)': times,
                                      'pca': pcas,
                                      'ace': aces,
                                      'mid': no3_means,
                                      'se': no3_standard_errors,})

no2_stat_df = pd.DataFrame.from_dict({'strain': strains,
                                      'Time (hours)': times,
                                      'pca': pcas,
                                      'ace': aces,
                                      'mid': no2_means,
                                      'se': no2_standard_errors,})

ace_stat_df = pd.DataFrame.from_dict({'strain': strains,
                                      'Time (hours)': times,
                                      'pca': pcas,
                                      'ace': aces,
                                      'mid': ace_means,
                                      'se': ace_standard_errors,})

no3_stat_df['low'] = no3_stat_df['mid'] - 1.96 * no3_stat_df['se']
no3_stat_df['high'] = no3_stat_df['mid'] + 1.96 * no3_stat_df['se']

no2_stat_df['low'] = no2_stat_df['mid'] - 1.96 * no2_stat_df['se']
no2_stat_df['high'] = no2_stat_df['mid'] + 1.96 * no2_stat_df['se']

ace_stat_df['low'] = ace_stat_df['mid'] - 1.96 * ace_stat_df['se']
ace_stat_df['high'] = ace_stat_df['mid'] + 1.96 * ace_stat_df['se']

In [16]:
no3_stat_df

,strain,Time (hours),pca,ace,mid,se,low,high
0,C. portucalensis MBL,0.0,0 µM PCA,0,10.567800,0.139409,10.294558,10.841042
1,C. portucalensis MBL,0.0,0 µM PCA,10,10.227733,0.096935,10.037740,10.417727
2,C. portucalensis MBL,0.0,0 µM PCA,50,10.456767,0.064266,10.330805,10.582729
3,C. portucalensis MBL,0.0,200 µM PCAox,0,10.488133,0.087441,10.316750,10.659517
4,C. portucalensis MBL,0.0,200 µM PCAox,10,10.273867,0.030638,10.213816,10.333918
5,C. portucalensis MBL,0.0,200 µM PCAox,50,10.371233,0.056511,10.260472,10.481995
6,C. portucalensis MBL,0.0,200 µM PCAred,0,10.385700,0.074868,10.238960,10.532440
7,C. portucalensis MBL,0.0,200 µM PCAred,10,10.322467,0.061947,10.201051,10.443883
8,C. portucalensis MBL,0.0,200 µM PCAred,50,10.392467,0.060903,10.273097,10.511836
9,C. portucalensis MBL,8.0,0 µM PCA,0,10.541967,0.120661,10.305472,10.778461


In [17]:
no2_stat_df

,strain,Time (hours),pca,ace,mid,se,low,high
0,C. portucalensis MBL,0.0,0 µM PCA,0,0.064233,0.007888,0.048772,0.079694
1,C. portucalensis MBL,0.0,0 µM PCA,10,0.163100,0.002076,0.159031,0.167169
2,C. portucalensis MBL,0.0,0 µM PCA,50,0.168767,0.003950,0.161025,0.176508
3,C. portucalensis MBL,0.0,200 µM PCAox,0,0.080100,0.001768,0.076634,0.083566
4,C. portucalensis MBL,0.0,200 µM PCAox,10,0.164933,0.001559,0.161877,0.167990
5,C. portucalensis MBL,0.0,200 µM PCAox,50,0.167200,0.004290,0.158792,0.175608
6,C. portucalensis MBL,0.0,200 µM PCAred,0,0.040433,0.005653,0.029353,0.051514
7,C. portucalensis MBL,0.0,200 µM PCAred,10,0.175467,0.008805,0.158209,0.192724
8,C. portucalensis MBL,0.0,200 µM PCAred,50,0.166367,0.005726,0.155143,0.177590
9,C. portucalensis MBL,8.0,0 µM PCA,0,0.238900,0.003342,0.232350,0.245450


In [18]:
def plot_ic_bokeh(no3_df, no2_df):
    
    no3_grouped = no3_df.groupby('ace')
    no2_grouped = no2_df.groupby('ace')

    plots = []

    for g in zip(no3_grouped, no2_grouped):
        
        no3_g, no2_g = g
        
        ace = no3_g[0]
        no3_mini_df = no3_g[1]
        no2_mini_df = no2_g[1]

        fig = bokeh.plotting.figure(width=600, 
                                    height=400, 
                                    title=f'{ace} mM acetate',
                                    x_axis_label = 'Time (hrs)',
                                    y_axis_label = f'Nitrate or Nitrite (mM)')

        no3_mini_group = no3_mini_df.groupby('pca')
        no2_mini_group = no2_mini_df.groupby('pca')

        legend_items = []

        for i, mg in enumerate(zip(no3_mini_group, no2_mini_group)):
            
            no3_mg, no2_mg = mg
            
            pca = no3_mg[0]
            
            no3_mdf = no3_mg[1]
            no2_mdf = no2_mg[1]

            color = bokeh.palettes.Colorblind3[i]

            c = fig.circle(no3_mdf['Time (hours)'], no3_mdf['mid'], color=color, size=7, alpha=1)
            l = fig.line(no3_mdf['Time (hours)'], no3_mdf['mid'], color=color, line_width=2)
            
            s = fig.square(no2_mdf['Time (hours)'], no2_mdf['mid'], color=color, size=7, alpha=1)
            l2 = fig.line(no2_mdf['Time (hours)'], no2_mdf['mid'], color=color, line_width=2)

            legend_items.append((f"{pca} (NO3)", [l, c,]))
            legend_items.append((f"{pca} (NO2)", [l2, s]))

            no3_xs = no3_mdf['Time (hours)'].values
            no3_lows = no3_mdf['low'].values
            no3_highs = no3_mdf['high'].values

            no3_err_xs = []
            no3_err_ys = []

            for x, l, h in zip(no3_xs, no3_lows, no3_highs):
                no3_err_xs.append((x, x))
                no3_err_ys.append((l, h))

            no3_error = fig.multi_line(no3_err_xs, no3_err_ys, color='grey', line_width=1.5, alpha=1)
            
            no2_xs = no2_mdf['Time (hours)'].values
            no2_lows = no2_mdf['low'].values
            no2_highs = no2_mdf['high'].values

            no2_err_xs = []
            no2_err_ys = []

            for x, l, h in zip(no2_xs, no2_lows, no2_highs):
                no2_err_xs.append((x, x))
                no2_err_ys.append((l, h))

            no2_error = fig.multi_line(no2_err_xs, no2_err_ys, color='grey', line_width=1.5, alpha=1)


        legend = bokeh.models.Legend(items=legend_items)
        legend.click_policy = "hide"

        fig.add_layout(legend, 'right')
        
        fig.y_range = bokeh.models.Range1d(-0.5, 11)

        fig.legend.label_text_font_size = '12pt'
        fig.title.text_font_size = "14pt"

        fig.yaxis.axis_label_text_font_size = '12pt'
        fig.xaxis.axis_label_text_font_size = '12pt'
        fig.yaxis.major_label_text_font_size = '10pt'
        fig.xaxis.major_label_text_font_size = '10pt'

        fig.output_backend = 'svg'

        plots.append(fig)
        
    return(plots)

In [19]:
ic_plots = plot_ic_bokeh(no3_stat_df, no2_stat_df)

In [20]:
bokeh.io.show(ic_plots[0])

In [21]:
bokeh.io.export_svgs(ic_plots[0], filename='./plots/ion_chromatography.svg')

['./plots/ion_chromatography.svg']